![alt text](https://drive.google.com/uc?id=10IGq-3t8ihrx6uf-Nogo2YgxO5HJ__Tm)

In [0]:
!pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=692122fc79c3111bfaf5e25b0ce8414956d8ab7d1e9775a8d51acd738eec7264
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [0]:
import yfinance as yf #Worked as of 12/02/20
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

#for å regne ut beta
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as web
from scipy import stats
import seaborn as sns
import random

#sentiment analysis
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [0]:
def stock_sentiment(tic):
  """
  Bruker aksje nyheter fra routers til å gjennomføre en sentiment analyse
  :param tic: ticker i form av String
  :returns: Sentiment nummer. + betyr at man er positiv til aksjen, - betyr at man er negativ til aksjen. max 1, min -1
  :rtype: float

  """
  #Reuters brukes for å finne nyheter
  info_page = requests.get('https://www.reuters.com/companies/%s.OQ/key-developments'%(str(tic))).text
  soup = bs(info_page,'lxml')
  #Finner overskrifter og innholdet i nyhetene.
  overskrift = soup.find_all('h4')
  innhold = soup.find_all("p", class_="TextLabel__text-label___3oCVw TextLabel__gray___1V4fk TextLabel__serif___3lOpX MarketStoryItem-description-3kmoG")
  #Merger sammen innholdet
  full_text_soup = overskrift + innhold
  final_text = ''
  for i in full_text_soup:
    final_text += i.text
  return sia.polarity_scores(final_text)['compound']

In [0]:
#Scrap data / akkumulere all finansiell data
s_og_p500 = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies').text
soup = bs(s_og_p500,'lxml')
stock_ticker = []
company = []
industry = []
#beta = [] 
sentiment = []

for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
  table_data = items.find_all(['th','td'])
  try:
    stock_ticker.append(table_data[0].a.text)
    company.append(table_data[1].a.text)
    industry.append(table_data[4].text)
    #beta.append(yf.Ticker(table_data[0].a.text).info['beta'])
    sentiment.append(stock_sentiment(table_data[0].a.text))
  except:
    pass


data_to_df = {'Ticker': stock_ticker, 'Company Name': company, 'Industry' : industry, 'Sentiment Analysis' : sentiment} #'Beta' : beta
df = pd.DataFrame(data_to_df)
#pd.set_option('display.max_rows', None)

sorted_df = df.sort_values(by='Industry', ascending=True)
sorted_df
#dummy = sorted_df
#for test in sorted_df:
#  print(dummy[test])


NameError: ignored

### i.Discuss what beta represents and explain briefly how you would estimate it for each company (you do not need to estimate beta).

Beta($\beta$) måler volatiliteten til enkelt aktiva/aksje relativt til volatiliteten av hele markedet. Og er et av de mest brukte nøkkeltallene for å evaluerer den forventede avkastningen av en enkeltaksje. 
*Baserer valget av aksjer på hvilken industri de tilhører og ikke på noen andre faktorer som beta, shareholders equity, debt-to-equity ratio og andre viktige nøkkeltall.*

$$\beta = \frac{Cov(r_{a}, r_{b})}{Var(r_b)}$$ 

$r_{a} \rightarrow \text{avkastningen til en individuell aksje}$

$r_{b} \rightarrow \text{avkastningen i det totale markedet}$




In [0]:
portfolio = ['MAS', 'GE', 'BXP', 'MHK', 'EBAY', 'JBHT', 'DIS', 'NOW', 'XRAY', 'PHM','AMP', 'APH', 'PLD','AES','CTSH']

def get_returns_percent(portfo):
  """
  Regner ut prosent avkastning over 36 måneder. Prøver å replikere beta fra yh finance.
  :param portfol: liste med aksje tickere
  :returns: Prosent avkastning over 36 måneder
  :rtype: df

  """
  #S&P 500 benchmark
  benchmark_price = yf.download('SPY',start = '2017-02-01', end = '2020-03-01', interval='1mo')
  benchmark_ret = benchmark_price["Adj Close"].pct_change()[1:]
  #Cleaning for duplicates
  benchmark_ret = benchmark_ret[benchmark_ret != 0.0]
  
  portfolio_returns = {}

  for stock in portfo:
    price_data = yf.download(stock ,start = '2017-02-01', end = '2020-03-01', interval='1mo')
    price_data = price_data['Adj Close']
    ret_data = price_data.pct_change()[1:]
    ret_data = ret_data[ret_data != 0.0]
    portfolio_returns[stock] = ret_data
  

  df = pd.DataFrame(portfolio_returns)
  return df


def list_of_regplots(dataframe):
  #S&P 500 benchmark
  benchmark_price = yf.download('SPY',start = '2017-02-01', end = '2020-03-01', interval='1mo')
  benchmark_ret = benchmark_price["Adj Close"].pct_change()[1:]
  #Cleaning for duplicates
  benchmark_ret = benchmark_ret[benchmark_ret != 0.0]

  dummy = dataframe
  list_of_regressions = []
  for stock in dataframe:
    sns.regplot(benchmark_ret.values, dummy[stock].values)
  

  

  #g = sns.PairGrid(list_of_regressions, y_vars=["test0"], x_vars=['MAS', 'GE', 'BXP', 'MHK', 'EBAY', 'JBHT', 'DIS', 'NOW', 'XRAY', 'PHM','AMP', 'APH', 'PLD','AES','CTSH'], height=4)
  #g.map(sns.regplot, color=".3")    
  #g.set(ylim=(-1, 11), yticks=[0, 5, 10]);

   
  #regressions = dataframe.
  #g =  sns.PairGrid(dataframe, y_vars=["Benchmark Returns"], x_vars=[dataframe.columns.values])


#test = get_beta(portfolio)
g = sns.regplot(benchmark_ret.values, test['BXP'].values)
sns.regplot(benchmark_ret.values, test['NOW'].values)


#plots = list_of_regplots(test)
#list_of_regplots(test)



#datasett = test
#g = sns.PairGrid(datasett)
#g.map(plt.scatter)



#sns.regplot(benchmark_ret.values, ret_data.values)
#plt.xlabel("Benchmark Returns")
#plt.ylabel("Portfolio Returns")
#plt.title("Portfolio Returns vs Benchmark Returns")
#plt.show()



NameError: ignored

In [0]:
googl = yf.Ticker('GOOGL')
visa = yf.Ticker('V')
industii = yf.Ticker

In [0]:
print("Google beta: ", googl.info['beta'])
print("Visa beta: ", visa.info['beta'])
visa.info
googl.info

Google beta:  1.025336
Visa beta:  0.929532


{'52WeekChange': -0.117253065,
 'SandP52WeekChange': -0.15999967,
 'address1': '1600 Amphitheatre Parkway',
 'algorithm': None,
 'annualHoldingsTurnover': None,
 'annualReportExpenseRatio': None,
 'ask': 1122.2,
 'askSize': 1400,
 'averageDailyVolume10Day': 3907750,
 'averageVolume': 2137293,
 'averageVolume10days': 3907750,
 'beta': 1.025336,
 'beta3Year': None,
 'bid': 1117.44,
 'bidSize': 800,
 'bookValue': 292.651,
 'category': None,
 'circulatingSupply': None,
 'city': 'Mountain View',
 'companyOfficers': [],
 'country': 'United States',
 'currency': 'USD',
 'dateShortInterest': 1582848000,
 'dayHigh': 1152.61,
 'dayLow': 1056.5,
 'dividendRate': None,
 'dividendYield': None,
 'earningsQuarterlyGrowth': 0.193,
 'enterpriseToEbitda': 13.426,
 'enterpriseToRevenue': 3.993,
 'enterpriseValue': 646251741184,
 'exDividendDate': None,
 'exchange': 'NMS',
 'exchangeTimezoneName': 'America/New_York',
 'exchangeTimezoneShortName': 'EDT',
 'expireDate': None,
 'fiftyDayAverage': 1376.1149,


In [0]:
my_portfolio = []

sorted_df.sample(n=15)





In [0]:
#Regner ut beta for google
price_data = yf.download('GOOGL',start = '2017-02-01', end = '2020-03-01', interval='1mo')
price_data = price_data['Adj Close']

#Kalkulerer den prosentvise forandringer fra 1 periode til den neste.
ret_data = price_data.pct_change()[1:]

#Clean for duplicates
ret_data = ret_data[ret_data != 0.0]


print(ret_data)

In [0]:
#S&P 500 benchmark
benchmark_price = yf.download('SPY',start = '2017-02-01', end = '2020-03-01', interval='1mo')
benchmark_ret = benchmark_price["Adj Close"].pct_change()[1:]

#Cleaning for duplicates
benchmark_ret = benchmark_ret[benchmark_ret != 0.0]
print(benchmark_ret)



In [0]:

sns.regplot(benchmark_ret.values, ret_data.values)
plt.xlabel("Benchmark Returns")
plt.ylabel("Portfolio Returns")
plt.title("Portfolio Returns vs Benchmark Returns")
plt.show()



In [0]:
(beta, alpha) = stats.linregress(benchmark_ret.values, ret_data.values)[0:2]
print("Beta is", round(beta, 6))
beta2 = stats.linregress(benchmark_ret.values, ret_data.values)[0]
print("Beta is", round(beta2[0], 6))

In [0]:
googl.info['beta'] #Difference = 0,00547

In [0]:
#Test two
price_data2 = yf.download('AAPL',start = '2017-03-01', end = '2020-03-01', interval='1mo')
price_data2 = price_data2['Adj Close']
#Daily returns
ret_data2 = price_data2.pct_change()[1:]

#Clean for duplicates
ret_data2 = ret_data2[ret_data2 != 0.0]

In [0]:
aapl = yf.Ticker('AAPL')
print("Beta til apple: ", aapl.info['beta'])

In [0]:
(beta, alpha) = stats.linregress(benchmark_ret.values, ret_data2.values)[0:2]
print("Beta is", round(beta, 6)) #difference 0.08165

NameError: ignored

In [0]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

sad_txt = '''
The coronavirus outbreak has radiated to at least 77 countries, killing more than 3,200 people and infecting more than 93,000. It has paralyzed cities and towns, disrupted business, travel and schools. But no place has seen more devastation than China, where the overwhelming majority of deaths and infections have occurred.

The New York Times asked Chinese readers all over the world to share their views on how the country responded to the coronavirus outbreak, which originated in Hubei Province in central China. We heard from readers in Europe, Australia, China and the United States. One was living just miles from the market in Wuhan that is considered by many experts to be ground zero for the virus.

Most expressed serious disappointment in how the Chinese government has handled the crisis. But others argued China, as a developing nation, has responded effectively. Here are some of their stories, edited and condensed for clarity.
'''

happy_txt = '''
Once upon a time, in a happy, happy place, there were happy people who lived in love and harmony. Each morning, friends and families would gather and sit quietly together appreciating the opportunity to spend another day with each other in such a happy place.

As they sat in communal happiness, they would breathe in unity, everybody breathing in together and then breathing out together. As the extended families breathed together they would feel their happiness swelling in their hearts, filling them to the brink of bursting. The slow, steady breathing helped their hearts expand as the joy expanded. Like waves lapping on the beach, as they breathed in their joy would mount into a rolling crest, and then, as they breathed out, it would gently wash up onto the sand and fall back into the ocean.

'''

print("Score sad_txt: ", sia.polarity_scores(sad_txt)['compound'])
print("Score happy_txt: ", sia.polarity_scores(happy_txt)['compound'])


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
Score sad_txt:  -0.8965
Score happy_txt:  0.9946


In [0]:
#test_list = ['fb', 'appl', 'googl']

#info_page = requests.get('https://www.reuters.com/companies/{}.OQ/key-developments'.format(test_list[0])).text
#soup = bs(info_page,'lxml')

#overskrift = soup.find_all('h4')
#innhold = soup.find_all("p", class_="TextLabel__text-label___3oCVw TextLabel__gray___1V4fk TextLabel__serif___3lOpX MarketStoryItem-description-3kmoG")

#texxt = overskrift + innhold
#final_text = ''
#for i in texxt:
#  final_text += i.text

#print(final_text)

#print("Score google: ", sia.polarity_scores(final_text)['compound'])


#print(soup.prettify())

#for i in test_list:
  #info_page = requests.get('https://www.reuters.com/companies/{}.OQ/key-developments'.format(i)).text
  #soup = bs(info_page,'lxml')

#def stock_sentiment(tic):
  #Reuters brukes for å finne nyheter
  #info_page = requests.get('https://www.reuters.com/companies/{}.OQ/key-developments'.format(tic)).text
  #soup = bs(info_page,'lxml')
  #Finner overskrifter og innholdet i nyhetene.
  #overskrift = soup.find_all('h4')
  #innhold = soup.find_all("p", class_="TextLabel__text-label___3oCVw TextLabel__gray___1V4fk TextLabel__serif___3lOpX MarketStoryItem-description-3kmoG")
  #Merger sammen innholdet
  #full_text_soup = overskrift + innhold
  #final_text = ''
  #for i in texxt:
    #final_text += i.text
  #return sia.polarity_scores(final_text)['compound']

#print('google', type(stock_sentiment('googl')))


In [0]:
for i in sentiment:
  print(i)

0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689
0.8689

In [0]:
lul = []
for i in stock_ticker:
  lul.append(stock_sentiment(i))

KeyboardInterrupt: ignored

In [0]:
#stock_sentiment(stock_ticker[2])
stock_sentiment('tic')

0.8689

In [0]:
stock_sentiment(stock_ticker[2])

0.8689

In [0]:
stock_sentiment('AMZN')

0.995

In [0]:
sentiment

[0.0,
 0.0,
 0.0,
 0.9974,
 0.0,
 0.9968,
 0.9973,
 0.9954,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9975,
 0.0,
 0.0,
 0.0,
 0.9982,
 0.9867,
 0.0,
 0.0,
 0.0,
 0.9992,
 0.0,
 0.7506,
 0.7506,
 0.0,
 0.995,
 0.0,
 0.0,
 -0.9916,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9975,
 0.0,
 0.9901,
 0.9992,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.886,
 0.9938,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9994,
 0.9995,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.999,
 0.0,
 0.0,
 0.9992,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9975,
 0.0,
 0.0,
 0.9945,
 0.0,
 0.9994,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9979,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9992,
 0.0,
 0.0,
 0.8602,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9975,
 0.996,
 0.9887,
 0.0,
 0.0,
 0.9991,
 0.0,
 0.9915,
 0.0,
 0.0,
 0.9982,
 0.0,
 0.989,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9964,
 0.0,
 0.0,
 0.996,
 0.0,
 0.0,
 0.9961,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9987,
 0.0,
 0.999,
 0.0,
 0.0,
 

In [0]:
sorted_df = df.sort_values(by='Industry', ascending=True)
sorted_df

,Ticker,Company Name,Industry,Sentiment Analysis
255,IPG,Interpublic Group,Advertising,0.0000
359,OMC,Omnicom Group,Advertising,0.0000
296,LMT,Lockheed Martin Corp.,Aerospace & Defense,0.0000
442,TXT,Textron Inc.,Aerospace & Defense,0.0000
243,HII,Huntington Ingalls Industries,Aerospace & Defense,0.0000
210,GD,General Dynamics,Aerospace & Defense,0.0000
73,BA,Boeing Company,Aerospace & Defense,0.0000
463,UTX,United Technologies,Aerospace & Defense,0.0000
53,ARNC,Arconic Inc.,Aerospace & Defense,0.0000
396,RTN,Raytheon Co.,Aerospace & Defense,0.0000


In [0]:

num1 = 'NFLX'
s_og_p500 = requests.get('https://finance.yahoo.com/quote/{0}/community?p={1}'.format(num1, num1)).text
https://finance.yahoo.com/quote/NFLX/community?p=NFLX
soup = bs(s_og_p500,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="NoJs featurephone" id="atomic" lang="en-US">
 <head prefix="og: http://ogp.me/ns#">
  <script>
   window.performance && window.performance.mark && window.performance.mark('PageStart');
  </script>
  <meta charset="utf-8"/>
  <title>
   Netflix, Inc. (NFLX) Stock Price, Quote, History &amp; News - Yahoo Finance
  </title>
  <meta content="NFLX, Netflix, Inc., NFLX stock chart, Netflix, Inc. stock chart, stock chart, stocks, quotes, finance" name="keywords"/>
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <meta content="on" property="twitter:dnt"/>
  <meta content="90376669494" property="fb:app_id"/>
  <meta content="#400090" name="theme-color"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="Find the latest Netflix, Inc. (NFLX) stock quote, history, news and other vital information to help you with your stock trading and investing." lang="en-US" name="description"/>
  <meta content="guce.yahoo.com" name